In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

import pickle
from tqdm import tqdm
import gc
from pathlib import Path

In [ ]:
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
# sys.path.append("/content/drive/MyDrive/HM/") # path to the `src`` folder
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()

In [ ]:
from src.data import DataHelper
from src.data.metrics import map_at_k, hr_at_k, recall_at_k

from src.utils import (
    calc_valid_date,
    merge_week_data,
    reduce_mem_usage,
    calc_embd_similarity,
)

In [ ]:
tqdm.pandas()

In [ ]:
data_dir = Path("data")
model_dir = Path("models")

Pepare data: encoding ids and preprocessing

In [ ]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thangndk67/h-and-m-csv-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/thangndk67/h-and-m-csv-dataset/versions/3


In [ ]:
dh = DataHelper(data_dir)

In [ ]:
# data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved

In [ ]:
data = dh.load_data(name="encoded_full", load_path="/workspaces/H-M-Fashion-RecSys/notebooks/data/processed/encoded_full")

In [ ]:
print(data.keys())

dict_keys(['user', 'item', 'inter'])


### Blend

In [ ]:
# pred1_lgb_rank = pd.read_parquet(data_dir/"processed"/"large_rank_valid.pqt")
# pred1_lgb_binary = pd.read_parquet(data_dir/"processed"/"large_binary_valid.pqt")
# pred1_nn = pd.read_parquet(data_dir/"external"/"large_nn_valid.pqt")
# pred1_nn.rename(columns={'article_id':'prediction'},inplace=True)

In [ ]:
# pred2_lgb_rank = pd.read_parquet(data_dir/"processed"/"small_rank_valid.pqt")
pred2_lgb_binary = pd.read_parquet(data_dir/"processed"/"small_binary_valid.pqt")
# pred2_nn = pd.read_parquet(data_dir/"external"/"small_nn_valid.pqt")
# pred2_nn.rename(columns={'article_id':'prediction'},inplace=True)

In [ ]:
pred2_lgb_binary.head()

,customer_id,prediction,prob
0,1371961,99011,0.025136
1,1371961,102629,0.019345
2,1371961,103305,0.015638
3,1371961,97247,0.015305
4,1371961,94701,0.014277


In [ ]:
# pred1_lgb_binary = pred1_lgb_binary.merge(pred1_nn, on=['customer_id','prediction'], how='left')
# pred1_lgb_binary['prob'] = pred1_lgb_binary['prob_x'] + pred1_lgb_binary['prob_y']

# pred2_lgb_binary = pred2_lgb_binary.merge(pred2_nn, on=['customer_id','prediction'], how='left')
# pred2_lgb_binary['prob'] = pred2_lgb_binary['prob_x'] + pred2_lgb_binary['prob_y']

In [ ]:
# pred1_lgb_rank   = pred1_lgb_rank.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred1_lgb_binary = pred1_lgb_binary.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred2_lgb_rank   = pred2_lgb_rank.sort_values(by='prob', ascending=False).reset_index(drop=True)
pred2_lgb_binary = pred2_lgb_binary.sort_values(by='prob', ascending=False).reset_index(drop=True)

In [ ]:
# pred1_lgb_rank   = pred1_lgb_rank.groupby('customer_id')['prediction'].apply(list).reset_index()
# pred1_lgb_binary = pred1_lgb_binary.groupby('customer_id')['prediction'].apply(list).reset_index()
# pred2_lgb_rank   = pred2_lgb_rank.groupby('customer_id')['prediction'].apply(list).reset_index()
pred2_lgb_binary = pred2_lgb_binary.groupby('customer_id')['prediction'].apply(list).reset_index()

In [ ]:
pred2_lgb_binary.head()

,customer_id,prediction
0,81,[73340]
1,87,"[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]"


In [ ]:
# pred1_lgb_rank.rename(columns={'prediction':'large_rank'},inplace=True)
# pred1_lgb_binary.rename(columns={'prediction':'large_binary'},inplace=True)
# pred2_lgb_rank.rename(columns={'prediction':'small_rank'},inplace=True)
pred2_lgb_binary.rename(columns={'prediction':'small_binary'},inplace=True)

In [ ]:
pred2_lgb_binary.head() 

,customer_id,small_binary
0,81,[73340]
1,87,"[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]"


In [ ]:
# pred = (
#     pred1_lgb_rank.merge(pred1_lgb_binary, on=['customer_id'])
#                     .merge(pred2_lgb_rank, on=['customer_id'])
#                     .merge(pred2_lgb_binary, on=['customer_id'])
#                     )

pred = pred2_lgb_binary

In [ ]:
pred.head()

,customer_id,small_binary
0,81,[73340]
1,87,"[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]"


In [ ]:
def cust_blend(dt, W = [1,1,1,1]):
    #Create a list of all model predictions
    REC = []

    # Second Try
    # REC.append(dt['large_rank'])
    # REC.append(dt['large_binary'])
    # REC.append(dt['small_rank'])
    REC.append(dt['small_binary'])

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return res[:12]

In [ ]:
pred['prediction'] = pred.progress_apply(cust_blend, W = [1.0, 1.3, 1.0, 1.3], axis=1)

100%|██████████| 56560/56560 [00:00<00:00, 78229.98it/s]


In [ ]:
pred.head()

,customer_id,small_binary,prediction
0,81,[73340],[73340]
1,87,"[100229, 27906, 33869, 98607]","[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]","[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]","[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]","[71107, 104150, 104841, 88141, 99400]"


In [ ]:
label = pd.read_parquet(data_dir/"processed"/"Recall 1"/"week1_label.pqt")
label = pd.merge(label, pred, on="customer_id", how="left")

In [ ]:
label.head()

,customer_id,article_id,small_binary,prediction
0,81,[28968],[73340],[73340]
1,87,[87372],"[100229, 27906, 33869, 98607]","[100229, 27906, 33869, 98607]"
2,108,"[69712, 77257, 33873]","[13043, 2220, 61304, 61305, 61306, 92030]","[13043, 2220, 61304, 61305, 61306, 92030]"
3,118,[97392],NaN,NaN
4,180,"[102398, 98410, 74, 95785, 103797, 105104, 103...","[105181, 104987, 95217, 3511]","[105181, 104987, 95217, 3511]"


In [ ]:
pred.head()

,customer_id,small_binary,prediction
0,81,[73340],[73340]
1,87,"[100229, 27906, 33869, 98607]","[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]","[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]","[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]","[71107, 104150, 104841, 88141, 99400]"


In [ ]:
# map_at_k(label["article_id"], label["prediction"], k=12)

# map_at_k(label["article_id"], label["prediction"], k=12)

def ensure_list(x):
    if isinstance(x, (list, np.ndarray)):
        return x
    elif x is None:
        return []
    else:
        return [x]

# Áp dụng conversion
actual_processed = [ensure_list(x) for x in label["article_id"]]
predicted_processed = [ensure_list(x) for x in label["prediction"]]

# Gọi hàm với dữ liệu đã xử lý
result = map_at_k(actual_processed, predicted_processed, k=12)

# 0.032249032703345615

### Test

In [ ]:
# pred1_lgb_rank = pd.read_parquet(data_dir/"processed"/"large_rank_test.pqt")
# pred1_lgb_binary = pd.read_parquet(data_dir/"processed"/"large_binary_test.pqt")
# pred1_nn = pd.read_parquet(data_dir/"processed"/"large_nn_test.pqt")
# pred1_nn.rename(columns={'article_id':'prediction'},inplace=True)

In [ ]:
# pred1_lgb_rank = pred1_lgb_rank.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred1_lgb_rank = pred1_lgb_rank.drop_duplicates(['customer_id','prediction'])

# pred1_lgb_binary = pred1_lgb_binary.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred1_lgb_binary = pred1_lgb_binary.drop_duplicates(['customer_id','prediction'])

# pred1_nn = pred1_nn.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred1_nn = pred1_nn.drop_duplicates(['customer_id','prediction'])

In [ ]:
# pred1_lgb_binary = pred1_lgb_binary.sort_values(by=['customer_id','prediction']).reset_index(drop=True)
# pred1_nn = pred1_nn.sort_values(by=['customer_id','prediction']).reset_index(drop=True)

# * not using merge here to avoid memory error
# pred1_lgb_binary['prob2'] = pred1_lgb_binary['prob'] + pred1_nn['prob']
# pred1_lgb_binary = pred1_lgb_binary.sort_values(by='prob2', ascending=False).reset_index(drop=True)

In [ ]:
# pred1_lgb_rank   = pred1_lgb_rank.groupby('customer_id')['prediction'].progress_apply(list).reset_index()
# pred1_lgb_binary = pred1_lgb_binary.groupby('customer_id')['prediction'].progress_apply(list).reset_index()

In [ ]:
# pred1_lgb_rank.to_parquet(data_dir/"large_rank_test_new.pqt")
# pred1_lgb_binary.to_parquet(data_dir/"large_binary_test_new.pqt")

In [ ]:
# * ------------------------------------------------------------

In [ ]:
pred2_lgb_rank = pd.read_parquet(data_dir/"processed"/"small_rank_test.pqt")
# pred2_lgb_binary = pd.read_parquet(data_dir/"processed"/"small_binary_test.pqt")
# pred2_nn = pd.read_parquet(data_dir/"processed"/"small_nn_test.pqt")
# pred2_nn.rename(columns={'article_id':'prediction'},inplace=True)

In [ ]:
pred2_lgb_rank.head()

,customer_id,prediction,prob
0,685430,72489,0.020827
1,685430,77093,0.020652
2,685430,90083,0.020351
3,685430,85415,0.020156
4,685430,86775,0.016552


In [ ]:
pred2_lgb_binary.head()

,customer_id,small_binary,prediction
0,81,[73340],[73340]
1,87,"[100229, 27906, 33869, 98607]","[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]","[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]","[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]","[71107, 104150, 104841, 88141, 99400]"


In [ ]:
pred2_lgb_rank = pred2_lgb_rank.sort_values(by='prob', ascending=False).reset_index(drop=True)
pred2_lgb_rank = pred2_lgb_rank.drop_duplicates(['customer_id','prediction'])


# pred2_lgb_binary = pred2_lgb_binary.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred2_lgb_binary['prediction'] = pred2_lgb_binary['prediction'].apply(tuple)

# pred2_lgb_binary = pred2_lgb_binary.drop_duplicates(['customer_id','prediction'])

# pred2_nn = pred2_nn.sort_values(by='prob', ascending=False).reset_index(drop=True)
# pred2_nn = pred2_nn.drop_duplicates(['customer_id','prediction'])

In [ ]:
pred2_lgb_rank.head()   

,customer_id,prediction,prob
0,254467,69517,0.942006
1,1027516,103144,0.913072
2,1358477,90083,0.911964
3,1279034,85722,0.910897
4,254467,69510,0.894246


In [ ]:
# Chuyển list thành string sắp xếp được
pred2_lgb_binary['prediction_str'] = pred2_lgb_binary['prediction'].apply(lambda x: ','.join(map(str, x)))

# Sắp xếp theo customer_id và prediction string
pred2_lgb_binary = pred2_lgb_binary.sort_values(
    by=['customer_id', 'prediction_str']
).reset_index(drop=True)

# Xóa cột tạm nếu cần
pred2_lgb_binary = pred2_lgb_binary.drop('prediction_str', axis=1)# pred2_nn = pred2_nn.sort_values(by=['customer_id','prediction']).reset_index(drop=True)

# pred2_lgb_binary['prob2'] = pred2_lgb_binary['prob'] + pred2_nn['prob']
# pred2_lgb_binary = pred2_lgb_binary.sort_values(by='prob', ascending=False).reset_index(drop=True)

In [ ]:
pred2_lgb_binary.head()

,customer_id,small_binary,prediction
0,81,[73340],[73340]
1,87,"[100229, 27906, 33869, 98607]","[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]","[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]","[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]","[71107, 104150, 104841, 88141, 99400]"


In [ ]:
pred2_lgb_rank   = pred2_lgb_rank.groupby('customer_id')['prediction'].progress_apply(list).reset_index()
pred2_lgb_binary = pred2_lgb_binary.groupby('customer_id')['prediction'].progress_apply(list).reset_index()

100%|██████████| 56560/56560 [00:00<00:00, 61580.28it/s]


In [ ]:
pred2_lgb_binary.head()

,customer_id,prediction
0,81,[[73340]]
1,87,"[[100229, 27906, 33869, 98607]]"
2,108,"[[13043, 2220, 61304, 61305, 61306, 92030]]"
3,180,"[[105181, 104987, 95217, 3511]]"
4,331,"[[71107, 104150, 104841, 88141, 99400]]"


In [ ]:
pred2_lgb_rank.head()

,customer_id,prediction
0,1,[16024]
1,2,[87146]
2,3,[78504]
3,4,[61176]
4,5,"[60764, 101368, 77916, 104813]"


In [ ]:
pred2_lgb_rank.shape

(1203314, 2)

In [ ]:
pred2_lgb_rank.to_parquet(data_dir/"small_rank_test.pqt")
# pred2_lgb_binary.to_parquet(data_dir/"small_binary_test.pqt")

In [ ]:
# ----------------------------------------------------------

In [ ]:
# pred1_lgb_rank.rename(columns={'prediction':'large_rank'},inplace=True)
# pred1_lgb_binary.rename(columns={'prediction':'large_binary'},inplace=True)
pred2_lgb_rank.rename(columns={'prediction':'small_rank'},inplace=True)
pred2_lgb_binary.rename(columns={'prediction':'small_binary'},inplace=True)

In [ ]:
pred2_lgb_binary.head()

,customer_id,small_binary
0,81,[[73340]]
1,87,"[[100229, 27906, 33869, 98607]]"
2,108,"[[13043, 2220, 61304, 61305, 61306, 92030]]"
3,180,"[[105181, 104987, 95217, 3511]]"
4,331,"[[71107, 104150, 104841, 88141, 99400]]"


In [ ]:
pred2_lgb_rank.head()

,customer_id,small_rank
0,1,[16024]
1,2,[87146]
2,3,[78504]
3,4,[61176]
4,5,"[60764, 101368, 77916, 104813]"


In [ ]:
pred2_lgb_binary['small_binary'] = pred2_lgb_binary['small_binary'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x
)

In [ ]:
pred2_lgb_binary.head()

,customer_id,small_binary
0,81,[73340]
1,87,"[100229, 27906, 33869, 98607]"
2,108,"[13043, 2220, 61304, 61305, 61306, 92030]"
3,180,"[105181, 104987, 95217, 3511]"
4,331,"[71107, 104150, 104841, 88141, 99400]"


In [ ]:
# pred = pred2_lgb_rank.merge(pred2_lgb_binary, on=['customer_id'], how='left')\
#                     .merge(pred1_lgb_rank, on=['customer_id'], how='left')\
#                     .merge(pred1_lgb_binary, on=['customer_id'], how='left')

pred = pred2_lgb_rank.merge(pred2_lgb_binary, on=['customer_id'], how='left')

In [ ]:
pred.head()

,customer_id,small_rank,small_binary
0,1,[16024],NaN
1,2,[87146],NaN
2,3,[78504],NaN
3,4,[61176],NaN
4,5,"[60764, 101368, 77916, 104813]",NaN


In [ ]:
pred['small_binary'].isna().sum()

np.int64(1147526)

In [ ]:
# for f in ['large_rank','large_binary']:
#     pred[f] = pred.progress_apply(lambda x:x[f] if not pd.isna(np.array(x[f])).any() else [], axis=1)

In [ ]:
# def cust_blend(dt, W = [1,1]):
#     #Create a list of all model predictions
#     REC = []

#     # Second Try
#     # REC.append(dt['large_rank'])
#     # REC.append(dt['large_binary'])
#     # REC.append(dt['small_rank'])
#     REC.append(dt['small_binary'])

#     #Create a dictionary of items recommended.
#     #Assign a weight according the order of appearance and multiply by global weights
#     res = {}
#     for M in range(len(REC)):
#         for n, v in enumerate(REC[M]):
#             if v in res:
#                 res[v] += (W[M]/(n+1))
#             else:
#                 res[v] = (W[M]/(n+1))

#     # Sort dictionary by item weights
#     res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

#     # Return the top 12 items only
#     return res[:12]
def cust_blend(dt, W=[1]):
    """
    Hàm blend đơn giản chỉ xử lý cột small_binary
    """
    # Lấy items từ small_binary
    items = dt['small_binary']
    
    # Khởi tạo dictionary kết quả
    res = {}
    
    # Xử lý items
    if isinstance(items, list):
        if len(items) == 0:
            return []  # Trả về list rỗng nếu không có gì
        
        # Nếu là list trong list
        if isinstance(items[0], list):
            items = items[0]
    elif items is None or (isinstance(items, float) and pd.isna(items)):
        return []
    elif isinstance(items, (int, float, np.integer)):
        items = [int(items)]
    else:
        # Cố gắng chuyển sang list
        try:
            items = list(items)
        except:
            return []
    
    # Tính toán weights
    for n, v in enumerate(items):
        if v in res:
            res[v] += (W[0] / (n + 1))
        else:
            res[v] = (W[0] / (n + 1))
    
    # Sắp xếp và trả về top 12
    sorted_items = sorted(res.items(), key=lambda x: -x[1])
    result = [item[0] for item in sorted_items[:12]]
    
    return result

In [ ]:
pred.head()

,customer_id,small_rank,small_binary
0,1,[16024],NaN
1,2,[87146],NaN
2,3,[78504],NaN
3,4,[61176],NaN
4,5,"[60764, 101368, 77916, 104813]",NaN


In [ ]:
pred['prediction'] = pred.progress_apply(cust_blend, W = [1.0, 1.3, 1.0, 1.3], axis=1) # , 1.0, 1.2

100%|██████████| 1203314/1203314 [00:07<00:00, 159173.56it/s]


In [ ]:
idx2uid = pickle.load(open(data_dir/"index_id_map/user_index2id.pkl", "rb"))
idx2iid = pickle.load(open(data_dir/"index_id_map/item_index2id.pkl", "rb"))

In [ ]:
def parse(x):
    l = ['0'+str(idx2iid[i]) for i in x]
    l = ' '.join(l[:12])
    return l

In [ ]:
pred['prediction'] = pred['prediction'].progress_apply(lambda x: parse(x))

100%|██████████| 1203314/1203314 [00:01<00:00, 1077375.06it/s]


In [ ]:
uid2idx = pickle.load(open(data_dir/"index_id_map/user_id2index.pkl", "rb"))
submission = pd.read_csv(f'{path}/sample_submission.csv')
submission['customer_id'] = submission['customer_id'].map(uid2idx)

In [ ]:
del submission['prediction']
submission = submission.merge(pred, on='customer_id', how='left')
submission['customer_id'] = submission['customer_id'].map(idx2uid)

In [ ]:
submission = submission[['customer_id', 'prediction']]

In [ ]:
submission.to_csv('large_recall_binary.csv', index=False)

In [ ]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,


In [ ]:
print(submission['prediction'].isnull().sum())

168666


In [ ]:
submission['prediction'] = submission['prediction'].apply(
    lambda x: [11565003] if (x is None or (isinstance(x, float) and pd.isna(x)) or x == []) else x
)

In [ ]:
print(submission['prediction'].isnull().sum())

0


In [ ]:
len(submission)

1371980

In [ ]:
del submission